<a href="https://colab.research.google.com/github/WaksKevin/SautiFi-notebooks/blob/main/SautiFi_draft_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources

- Base Path to audio clips (Instead of downloading the whole dataset which is huge, or loading it from HuggingFace which will take a lot of time, the assets have been put to the server and can be accessed via a link (an endpoint)). The rest of resources are in  kaggle input directory '/kaggle/input/'

In [1]:
clips_path = "https://sautifi.duckdns.org/assets/data/clips"

- Example of an audio clip - https://sautifi.duckdns.org/assets/data/clips/common_voice_sw_29525813.mp3

# Install libraries and packages

In [ ]:
%pip install --upgrade pip
%pip install pandas requests librosa transformers torchaudio ipywebrtc

# Loading the model and processor

In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import os

In [4]:
model_path = "models/wav2vec2_swahili_model"
processor_path = "models/wav2vec2_swahili_processor"

if not (os.path.exists(model_path) and os.path.exists(processor_path)):
    print("Saving model locally so that it doesn't redownload when run later")
    os.makedirs(model_path)
    os.makedirs(processor_path)

    processor = Wav2Vec2Processor.from_pretrained("Adeptschneider/wav2vec-large-swahili-asr-model-with-swahili-language-model")
    model = Wav2Vec2ForCTC.from_pretrained("Adeptschneider/wav2vec-large-swahili-asr-model-with-swahili-language-model")

    processor.save_pretrained(processor_path)
    model.save_pretrained(model_path)
else:
    print("Using locally saved model")
    processor = Wav2Vec2Processor.from_pretrained(processor_path)
    model = Wav2Vec2ForCTC.from_pretrained(model_path)

file_path = os.path.abspath(model_path)
print(file_path)



Saving model locally so that it doesn't redownload when run later


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at Adeptschneider/wav2vec-large-swahili-asr-model-with-swahili-language-model were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Adeptschneider/wav2vec-large-swahili-asr-model-with-swahili-language-model and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder

/content/models/wav2vec2_swahili_model


# Example Transcriptions

In [5]:
from io import BytesIO
import IPython.display as ipd
import librosa
import requests
import torch

In [6]:
# Define the URL of the audio file
audio_url = f"{clips_path}/common_voice_sw_29525813.mp3"

# Download the audio file
response = requests.get(audio_url)

# Convert the audio content to bytes using BytesIO
audio_bytes = BytesIO(response.content)

# Load the audio data from bytes using librosa
audio_data, _ = librosa.load(audio_bytes, sr=16000)

# Create audio widget
audio_widget = ipd.Audio(audio_data, rate=16000)

# Display the audio widget
ipd.display(audio_widget)

# Tokenize the input audio
input_values = processor(audio_data, sampling_rate=16000, return_tensors="pt").input_values

# Make the inference
with torch.no_grad():
    logits = model(input_values).logits

# Identify the tokens with the highest probability as the predicted tokens
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted ids to text
transcription = processor.batch_decode(predicted_ids)[0]

print(transcription)

nchi hizo mbili zilikuwa na kipaumbele juu ya mataifa mengine ya milki hii kubwa


In [7]:
# Define the URL of the audio file
audio_url = f"{clips_path}/common_voice_sw_29525815.mp3"

# Download the audio file
response = requests.get(audio_url)

# Convert the audio content to bytes using BytesIO
audio_bytes = BytesIO(response.content)

# Load the audio data from bytes using librosa
audio_data, _ = librosa.load(audio_bytes, sr=16000)

# Create audio widget
audio_widget = ipd.Audio(audio_data, rate=16000)

# Display the audio widget
ipd.display(audio_widget)

# Tokenize the input audio
input_values = processor(audio_data, sampling_rate=16000, return_tensors="pt").input_values

# Make the inference
with torch.no_grad():
    logits = model(input_values).logits

# Identify the tokens with the highest probability as the predicted tokens
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted ids to text
transcription = processor.batch_decode(predicted_ids)[0]

print(transcription)

wataalamu kutoka nchi nyingi nyingi walianza kukusanya pesa kuhamisho wa majengo


# Test using you own voice

In [8]:
from google.colab import output
output.enable_custom_widget_manager()

from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

- Load recorging widget

In [9]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

- Get transcript of recording

In [16]:
# Record audio first
try:
    with open('recording.webm', 'wb') as f:
        f.write(recorder.audio.value)
    !ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic

    # Load the audio data from bytes using librosa
    audio_data, _ = librosa.load("file.wav", sr=16000)

    # Tokenize the input audio
    input_values = processor(audio_data, sampling_rate=16000, return_tensors="pt").input_values

    # Make the inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Identify the tokens with the highest probability as the predicted tokens
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the predicted ids to text
    transcription = processor.batch_decode(predicted_ids)[0]

    print(transcription)
except:
    pass


habari yako
